# Парсинг данных с JSON файла

In [2]:
import json
import glob
import pandas as pd
from datetime import datetime
from IPython.display import display

## Парсинг

In [4]:
all_jsons = glob.glob(r"C:\Users\User\Desktop\Data\*.json")
Sum_obj = []

for file_path in all_jsons:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            data['file_source'] = file_path
            Sum_obj.append(data)
    except Exception as e:
        print(f"Ошибка при загрузке файла {file_path}: {str(e)}")    

## Посмотрим количество файлов

In [6]:
len(Sum_obj)

14

## Извлечение данных из файлов

In [8]:
def parse_custom_date(day, month, time):
    month_map = {
        'января': '01', 'февраля': '02', 'марта': '03', 'апреля': '04',
        'мая': '05', 'июня': '06', 'июля': '07', 'августа': '08',
        'сентября': '09', 'октября': '10', 'ноября': '11', 'декабря': '12'
    }
    current_year = datetime.now().year
    month_num = int(month_map.get(month, '01'))
    year = current_year if month_num <= datetime.now().month else current_year - 1
    
    try:
        date_str = f"{day} {month_map.get(month, '01')} {year} {time}"
        return datetime.strptime(date_str, "%d %m %Y %H:%M")
    except (ValueError, KeyError):
        return None


def process_data(data_list):
    articles = []
    content_type_mapping = {
        'хакатон': 'хакатон',
        'премия': 'награждение',
        'исследование': 'исследование'
    }
    keywords_list = ['киберспорт', 'хакатон', 'премия', 
                    'искусственный интеллект', 'разработка', 'технологии']
    
    for data in data_list:
        refs = data.get('refs', []) or []
        
        for ref in refs:
            if not isinstance(ref, list) or len(ref) < 3:
                continue
                
            text = ref[0] if ref[0] else ""
            date_info = ref[1] if isinstance(ref[1], dict) else {'day': '1', 'month': 'января', 'time': '00:00'}
            stats = ref[2] if isinstance(ref[2], dict) else {'rate': 0, 'views': 0}
            
            text_lower = text.lower()
            keywords = [word for word in keywords_list if word in text_lower]
            
            content_type = 'новость'
            for key, value in content_type_mapping.items():
                if key in text_lower:
                    content_type = value
                    break
                    
            shortened_text = text[:200] + '...' if len(text) > 200 else text
            
            articles.append({
                'text': shortened_text,
                'full_text': text,
                'date': parse_custom_date(
                    date_info.get('day', '1'),
                    date_info.get('month', 'января'),
                    date_info.get('time', '00:00')
                ),
                'rating': stats.get('rate', 0),
                'views': stats.get('views', 0),
                'keywords': ', '.join(keywords) if keywords else 'нет',
                'content_type': content_type,
                'source_file': data.get('file_source', 'Неизвестно')
            })
    
    return pd.DataFrame(articles)

## Создание датасета


In [10]:
articles_df = process_data(Sum_obj)

In [34]:
articles_df

,text,full_text,date,rating,views,keywords,content_type,source_file
0,7 декабря состоялась церемония награждения лау...,7 декабря состоялась церемония награждения лау...,2024-12-08 09:30:00,6,2300,"киберспорт, премия, технологии",награждение,C:\Users\User\Desktop\Data\Cybersport.Metarati...
1,7 декабря состоялась церемония награждения лау...,7 декабря состоялась церемония награждения лау...,2024-12-08 09:30:00,6,2300,"киберспорт, премия, технологии",награждение,C:\Users\User\Desktop\Data\ESforce Holding.json
2,Из совместного исследования издателя компьютер...,Из совместного исследования издателя компьютер...,2024-07-29 13:50:00,12,2800,киберспорт,исследование,C:\Users\User\Desktop\Data\ESforce Holding.json
3,По статистике The Esports Observer за I полови...,По статистике The Esports Observer за I полови...,2024-07-09 09:33:00,18,6200,киберспорт,новость,C:\Users\User\Desktop\Data\ESforce Holding.json
4,Компании SuperData Research и PayPal провели с...,Компании SuperData Research и PayPal провели с...,2025-04-11 12:43:00,11,10000,киберспорт,исследование,C:\Users\User\Desktop\Data\ESforce Holding.json
...,...,...,...,...,...,...,...,...
1107,"Привет, Хабр! Представляю вашему вниманию пере...","Привет, Хабр! Представляю вашему вниманию пере...",2025-01-30 10:43:00,32,9300,"разработка, технологии",исследование,C:\Users\User\Desktop\Data\Федерация креативны...
1108,Источник\n«Умный человек не делает все ошибки ...,Источник\n«Умный человек не делает все ошибки ...,2025-04-24 12:37:00,3,7700,технологии,новость,C:\Users\User\Desktop\Data\Федерация креативны...
1109,"Привет, Хабр! Сегодня мы решили поделиться с с...","Привет, Хабр! Сегодня мы решили поделиться с с...",2024-10-12 14:09:00,19,9300,"разработка, технологии",новость,C:\Users\User\Desktop\Data\Федерация креативны...
1110,"Привет! Меня зовут Никита Бокарев, я креативны...","Привет! Меня зовут Никита Бокарев, я креативны...",2024-09-04 16:44:00,52,52000,киберспорт,новость,C:\Users\User\Desktop\Data\Федерация креативны...


## Вывод результатов


In [13]:
print(f"=== Статистика по статьям ===")
print(f"Средний рейтинг: {articles_df['rating'].mean():.1f}")
print(f"Среднее количество просмотров: {articles_df['views'].mean():.0f}")
print(f"Медианное количество просмотров: {articles_df['views'].median():.0f}")

=== Статистика по статьям ===
Средний рейтинг: 23.9
Среднее количество просмотров: 21016
Медианное количество просмотров: 7000


In [38]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   text          1112 non-null   object        
 1   full_text     1112 non-null   object        
 2   date          1111 non-null   datetime64[ns]
 3   rating        1112 non-null   int64         
 4   views         1112 non-null   int64         
 5   keywords      1112 non-null   object        
 6   content_type  1112 non-null   object        
 7   source_file   1112 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 69.6+ KB


## Сделаем CSV файл из полученных данных

In [15]:
articles_df.to_csv('jsons_info.csv')